### Name: Syed Asad Rizvi

### ERP ID: 25365

### Project: Healthcare Q&A Bot Using OpenAI Word Embeddings

_______________________________________________________________________________________________________________________________

Importing libraries

In [2]:
import pandas as pd
import openai
from getpass import getpass
from youtube_transcript_api import YouTubeTranscriptApi
from openai.embeddings_utils import get_embedding, cosine_similarity

import warnings
warnings.filterwarnings('ignore')

Initializing OpenAI Models & API Key

In [10]:
# Text generation model
COMPLETIONS_MODEL = "text-davinci-003"

# Word embedding model
EMBEDDINGS_MODEL = "text-embedding-ada-002"

# Entering OpenAI API key
openai.api_key = getpass("Enter your OpenAI API Key")

Fetching YouTube Healthcare podcast episodes

In [3]:
# Flu & Covid
flu_covid_ep = YouTubeTranscriptApi.get_transcript("FMTQCsv-row&list=PL_OlobI2SUirmdJ68fTIBmkSsrQbr-0pY&index=25")

# Back pain 
back_pain_ep = YouTubeTranscriptApi.get_transcript("r7hf-9mac3Y&list=PL_OlobI2SUirmdJ68fTIBmkSsrQbr-0pY&index=37")

# Anxiety
anxiety_ep = YouTubeTranscriptApi.get_transcript("PIOWM1KunQ4&list=PL_OlobI2SUirmdJ68fTIBmkSsrQbr-0pY&index=38")

# Exercise
exercise_ep = YouTubeTranscriptApi.get_transcript("4cwSdRer9bU&list=PL_OlobI2SUirmdJ68fTIBmkSsrQbr-0pY&index=39")

# Better Sleep
better_sleep_ep = YouTubeTranscriptApi.get_transcript("8lLtuVUMmMc&list=PL_OlobI2SUirmdJ68fTIBmkSsrQbr-0pY&index=40")

# Stress & Burnouts
stress_burnout_ep = YouTubeTranscriptApi.get_transcript("OfeIOI8ov2A&list=PL_OlobI2SUirmdJ68fTIBmkSsrQbr-0pY&index=47")

# Head Migraine
head_mig_ep = YouTubeTranscriptApi.get_transcript("1IBPG58aizU&list=PL_OlobI2SUirmdJ68fTIBmkSsrQbr-0pY&index=48")

# Seasonal disorders
seasonal_dis_ep = YouTubeTranscriptApi.get_transcript("3Lj7XGQspbI&list=PL_OlobI2SUirmdJ68fTIBmkSsrQbr-0pY&index=55")

# Suicide & Depression
suicide_dep_ep = YouTubeTranscriptApi.get_transcript("xPVIw2IZxBA&list=PL_OlobI2SUirmdJ68fTIBmkSsrQbr-0pY&index=59")

# Diabetes
diabetes_ep = YouTubeTranscriptApi.get_transcript("3U-F1bWKbqQ&list=PL_OlobI2SUirmdJ68fTIBmkSsrQbr-0pY&index=66")

In [4]:
# Putting each transcript in a list

transcripts = [flu_covid_ep, back_pain_ep, anxiety_ep, exercise_ep, better_sleep_ep, stress_burnout_ep, head_mig_ep,
              seasonal_dis_ep, suicide_dep_ep, diabetes_ep]

for transcript in transcripts:
    # loop through each dictionary in the transcript and convert the 'start' values to integers
    for caption in transcript:
        caption['start'] = int(caption['start'])

Loading the Excel sheets and sorting 'start' and 'end' values with their respective texts in each episode

In [5]:
# Load the excel file
qna_bot = pd.read_excel('Healthcare Bot.xlsx', sheet_name=None)

k = 0
# iterate through each sheet of the DataFrame
for each_sheet in qna_bot:
    for i, row in qna_bot[each_sheet].iterrows():
        text = [t['text'] for t in transcripts[k] if t['start'] >= row['start'] and t['start'] <= row['end']]
        context = ' '.join(text)
        qna_bot[each_sheet].at[i, 'context'] = context
    k = k+1

Creating a dataframe for each episode separately

In [6]:
for sheet_name in qna_bot.keys():
    df = qna_bot[sheet_name]
    df = df.drop('Unnamed: 0', axis=1)
    qna_bot[sheet_name] = df

Merging the dataframes of all episodes into a single dataframe

In [7]:
# Create an empty list to store all the dataframes
dfs = []

# Loop through each sheet in the qna_bot dictionary and append its dataframe to the list
for sheet_name in qna_bot.keys():
    dfs.append(qna_bot[sheet_name])

# Concatenate all the dataframes in the list into a single dataframe
merged_df = pd.concat(dfs, ignore_index=True)

In [8]:
merged_df

,episode,url,start,end,context
0,Episode 1,https://www.youtube.com/watch?v=FMTQCsv-row&li...,90,320,finally i feel like i i haven't seen you throu...
1,Episode 1,https://www.youtube.com/watch?v=FMTQCsv-row&li...,320,466,yeah absolutely um and i think what's interest...
2,Episode 1,https://www.youtube.com/watch?v=FMTQCsv-row&li...,466,744,yeah and i think that's the thing that we need...
3,Episode 1,https://www.youtube.com/watch?v=FMTQCsv-row&li...,744,895,got i'm going to get my flu shot my both of my...
4,Episode 1,https://www.youtube.com/watch?v=FMTQCsv-row&li...,895,960,maybe have not gotten the flu shot in their li...
...,...,...,...,...,...
133,Episode 10,https://www.youtube.com/watch?v=3U-F1bWKbqQ&li...,990,1447,want to talk about another thing and that is f...
134,Episode 10,https://www.youtube.com/watch?v=3U-F1bWKbqQ&li...,1447,1624,being utilized what what's the holdup here pro...
135,Episode 10,https://www.youtube.com/watch?v=3U-F1bWKbqQ&li...,1624,1712,first discovered in the 1920s is first deliver...
136,Episode 10,https://www.youtube.com/watch?v=3U-F1bWKbqQ&li...,1712,1763,in fact I have patients that choose to use vio...


Generating word embedding of each context and creating the column for the same

In [11]:
merged_df['embedding'] = merged_df['context'].apply(lambda row: get_embedding(row, engine='text-embedding-ada-002'))

In [12]:
merged_df

,episode,url,start,end,context,embedding
0,Episode 1,https://www.youtube.com/watch?v=FMTQCsv-row&li...,90,320,finally i feel like i i haven't seen you throu...,"[-0.0333162285387516, -0.00646474352106452, 0...."
1,Episode 1,https://www.youtube.com/watch?v=FMTQCsv-row&li...,320,466,yeah absolutely um and i think what's interest...,"[-0.030882058665156364, 0.0020020385272800922,..."
2,Episode 1,https://www.youtube.com/watch?v=FMTQCsv-row&li...,466,744,yeah and i think that's the thing that we need...,"[-0.01247295830398798, 0.011332061141729355, 0..."
3,Episode 1,https://www.youtube.com/watch?v=FMTQCsv-row&li...,744,895,got i'm going to get my flu shot my both of my...,"[-0.021822135895490646, 0.013645283877849579, ..."
4,Episode 1,https://www.youtube.com/watch?v=FMTQCsv-row&li...,895,960,maybe have not gotten the flu shot in their li...,"[-0.013866368681192398, 0.014302095398306847, ..."
...,...,...,...,...,...,...
133,Episode 10,https://www.youtube.com/watch?v=3U-F1bWKbqQ&li...,990,1447,want to talk about another thing and that is f...,"[-0.004034609533846378, -0.007231032010167837,..."
134,Episode 10,https://www.youtube.com/watch?v=3U-F1bWKbqQ&li...,1447,1624,being utilized what what's the holdup here pro...,"[0.004235747270286083, -0.0019685362931340933,..."
135,Episode 10,https://www.youtube.com/watch?v=3U-F1bWKbqQ&li...,1624,1712,first discovered in the 1920s is first deliver...,"[-0.01571071334183216, 0.013606120832264423, 0..."
136,Episode 10,https://www.youtube.com/watch?v=3U-F1bWKbqQ&li...,1712,1763,in fact I have patients that choose to use vio...,"[-0.018342748284339905, -0.0028241740074008703..."


Saving the model

In [13]:
import pickle

# Specify the file path where you want to save the pickle file
file_path = 'model.pkl'

# Save the DataFrame as a pickle file
with open(file_path, 'wb') as f:
    pickle.dump(merged_df, f)

Calculating cosine similarity of the question with the context and creating column for the same

In [192]:
# Example question
question = "How good is morning walk for a person's health?"

# Generating word embedding of the question asked 
question_vector = get_embedding(question, engine='text-embedding-ada-002')

# Calculating cosine similarity of question vector with the context embedding 
merged_df["similarities"] = merged_df['embedding'].apply(lambda x: cosine_similarity(x, question_vector))

# Sorting the cosine similarities in terms of highest and most similar context at top
sorted_merged_df = merged_df.sort_values("similarities", ascending=False).head(5)

sorted_merged_df

,episode,url,start,end,context,embedding,similarities
49,Episode 4,https://www.youtube.com/watch?v=4cwSdRer9bU&list=PL_OlobI2SUirmdJ68fTIBmkSsrQbr-0pY&index=39,151,235,both audiences today what do you think sure let's just start off with some of the facts how much exercise is really recommended for an adults on a weekly basis sure most of the institutional guidelines suggest that we do 30 minutes of moderate-intensity exercise 5 days a week and most of the health benefits attributed to exercise can be had from that volume of exercise so think 150 minutes of exercise accumulated over the course of the week there seems to be added benefit working up to about an hour a day of exercise ok so when you say that I'm thinking of the treadmills in the gym that's a 20 minute limit so are you talking about 30 minutes of cardio exercise or just exercise in general most of the conventional exercise recommendations are focused on aerobic exercise walking cycling swimming for example but it doesn't have to happen all at the same time there can be an accumulation of activity throughout the course of the day so imagine 10 minute walk before work in the morning 10 minute walk during a lunch break and it walk in the evening before or after dinner that's your thirty minutes so your exercise physiologist right what exactly is that and how is that different than being a trainer okay is it an exercise physiologist is supposed to know the the the physiologic effects of exercise on the human body and most,"[0.012411648407578468, 0.007749965880066156, 0.014446664601564407, -0.05099893733859062, -0.01690428890287876, 0.00869920663535595, -0.010818745009601116, -0.019010823220014572, -0.01116333156824112, 0.0007074607419781387, -0.013146335259079933, 0.03302837908267975, -0.012756235897541046, -0.00398876192048192, -0.009167325682938099, 0.012359635904431343, 0.04025821387767792, -0.0020268892403692007, 0.005594669375568628, -0.008010031655430794, 0.000450239225756377, 0.010896764695644379, -0.007847490720450878, -0.011403893120586872, -0.003975758794695139, 0.01962197758257389, 0.03063577227294445, -0.013094321824610233, 4.4571854232344776e-05, -0.032248180359601974, 0.011436400935053825, -0.011735477484762669, -0.00822458602488041, -0.010415642522275448, 0.0012255609035491943, 0.015694981440901756, -0.011410394683480263, 0.0121125727891922, 0.013627457432448864, -0.01258719339966774, 0.0036929368507117033, -0.012782243080437183, -0.009264850057661533, 0.004525147844105959, 0.00283309374935925, 0.008263596333563328, 0.013913529925048351, -0.02800910547375679, -0.0012873265659436584, 0.04098639637231827, 0.007847490720450878, -0.0025177637580782175, -0.017684485763311386, 0.0010110065340995789, -0.014095575548708439, 0.0037579534109681845, -0.023522967472672462, 0.0314679816365242, -0.002647796645760536, -0.013913529925048351, -0.005838481243699789, -0.009316863492131233, -0.019595971331000328, 0.009453398175537586, -0.017372407019138336, -0.002025263849645853, 0.007333860732614994, 0.012886269018054008, -0.005324850790202618, 0.0015238240594044328, 0.01012306846678257, 0.012697720900177956, 0.0006591047276742756, 0.0112283481284976, 0.03235220909118652, -0.036981381475925446, -0.00015177288150880486, -0.014537688344717026, -0.0021861796267330647, 0.008647194132208824, -0.007288348861038685, -0.020220128819346428, -0.005887243431061506, 0.028399204835295677, 0.03042771853506565, -0.007743464317172766, 0.0006176566821523011, -0.012301120907068253, -0.00674221059307456, 0.025304419919848442, 0.007457391824573278, -0.002524265320971608, 0.024810293689370155, 0.01973900757730007, 0.013666466809809208, 0.01096828281879425, -0.03128593787550926, 0.003063902258872986, 0.009752473793923855, -0.006885246839374304, ...]",0.809692
59,Episode 4,https://www.youtube.com/watch?v=4cwSdRer9bU&list=PL_OlobI2SUirmdJ68fTIBmkSsrQbr-0pY&index=39,1205,1425,what about heart rate is there a certain link is there a benefit to a heart rate goal so like during aerobic exercise yeah

In [127]:
pd.set_option('display.max_colwidth', None)

Merging the top most similar contexts obtained above and putting in a list

In [193]:
context = []
for i, row in sorted_merged_df.iterrows():
    context.append(row['context'])

context

["both audiences today what do you think sure let's just start off with some of the facts how much exercise is really recommended for an adults on a weekly basis sure most of the institutional guidelines suggest that we do 30 minutes of moderate-intensity exercise 5 days a week and most of the health benefits attributed to exercise can be had from that volume of exercise so think 150 minutes of exercise accumulated over the course of the week there seems to be added benefit working up to about an hour a day of exercise ok so when you say that I'm thinking of the treadmills in the gym that's a 20 minute limit so are you talking about 30 minutes of cardio exercise or just exercise in general most of the conventional exercise recommendations are focused on aerobic exercise walking cycling swimming for example but it doesn't have to happen all at the same time there can be an accumulation of activity throughout the course of the day so imagine 10 minute walk before work in the morning 10 m

Joining all contexts together for the prompt

In [194]:
text = "\n".join(context)
text

"both audiences today what do you think sure let's just start off with some of the facts how much exercise is really recommended for an adults on a weekly basis sure most of the institutional guidelines suggest that we do 30 minutes of moderate-intensity exercise 5 days a week and most of the health benefits attributed to exercise can be had from that volume of exercise so think 150 minutes of exercise accumulated over the course of the week there seems to be added benefit working up to about an hour a day of exercise ok so when you say that I'm thinking of the treadmills in the gym that's a 20 minute limit so are you talking about 30 minutes of cardio exercise or just exercise in general most of the conventional exercise recommendations are focused on aerobic exercise walking cycling swimming for example but it doesn't have to happen all at the same time there can be an accumulation of activity throughout the course of the day so imagine 10 minute walk before work in the morning 10 mi

In [195]:
context = text
context

"both audiences today what do you think sure let's just start off with some of the facts how much exercise is really recommended for an adults on a weekly basis sure most of the institutional guidelines suggest that we do 30 minutes of moderate-intensity exercise 5 days a week and most of the health benefits attributed to exercise can be had from that volume of exercise so think 150 minutes of exercise accumulated over the course of the week there seems to be added benefit working up to about an hour a day of exercise ok so when you say that I'm thinking of the treadmills in the gym that's a 20 minute limit so are you talking about 30 minutes of cardio exercise or just exercise in general most of the conventional exercise recommendations are focused on aerobic exercise walking cycling swimming for example but it doesn't have to happen all at the same time there can be an accumulation of activity throughout the course of the day so imagine 10 minute walk before work in the morning 10 mi

Setting the prompt and hyperparameters using OpenAI text model

In [196]:
prompt = f"""Answer the following question using only the context below. If you don't know the answer, just reply I don't know.

Context:
{context}

Q: {question}
A:"""

openai.Completion.create(prompt=prompt, temperature=1, max_tokens=500, top_p=1, frequency_penalty=0, presence_penalty=0, 
                         model=COMPLETIONS_MODEL)["choices"][0]["text"].strip(" \n")

'Morning walks are a great way to get the recommended 30 minutes of moderate-intensity exercise 5 days a week. They can help with improved sleep quality, stress management, reduced prevalence of heart disease, delayed age-related muscle loss, and even cancer prevention.'

-------------------------------------------xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx-------------------------------------------